<a href="https://colab.research.google.com/github/imarariyal/DSPL-ICW/blob/main/DSPL_ICW_DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random
import warnings
warnings.filterwarnings('ignore')